In [25]:
import warnings
warnings.filterwarnings("ignore")
from LLM_auto_label import get_response_from_prompt
from langchain_community.tools import TavilySearchResults
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from uqlm import BlackBoxUQ
from uqlm import UQEnsemble
from uqlm import LLMPanel
from uqlm.judges import LLMJudge
import os
from dotenv import load_dotenv
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
import pandas as pd
import tqdm

In [26]:
entity_name = "Steve Jobs"
dataset_src = "New Dataset/"
df = pd.read_csv(f"{dataset_src}{entity_name}.csv")
df.head()

,atomic_facts,label,search_results,prompt
0,Steve Jobs là một trong những nhà sáng lập và ...,Supported,[{'title': 'Steve Jobs – Wikipedia tiếng Việt'...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...
1,Steve Jobs nổi tiếng với vai trò đồng sáng lập...,Supported,[{'title': 'Hành trình đầy cảm hứng của doanh ...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...
2,Steve Jobs có sự ảnh hưởng sâu rộng đến ngành ...,Supported,"[{'title': 'Steve Jobs, người thay đổi cả thế ...",\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...
3,Steve Jobs sinh ngày 24 tháng 2 năm 1955 tại S...,Supported,"[{'title': 'Steve Jobs - Wikipedia', 'url': 'h...",\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...
4,Steve Jobs được một cặp vợ chồng người Mỹ gốc ...,Unsupported,[{'title': 'Steve Jobs – Wikipedia tiếng Việt'...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...


In [27]:
atomic_facts = df['atomic_facts'].tolist()
atomic_facts

['Steve Jobs là một trong những nhà sáng lập và lãnh đạo nổi bật nhất trong lịch sử ngành công nghệ.',
 'Steve Jobs nổi tiếng với vai trò đồng sáng lập Apple Inc.',
 'Steve Jobs có sự ảnh hưởng sâu rộng đến ngành công nghiệp máy tính và thiết bị di động.',
 'Steve Jobs sinh ngày 24 tháng 2 năm 1955 tại San Francisco, California, Hoa Kỳ.',
 'Steve Jobs được một cặp vợ chồng người Mỹ gốc Syri nhận nuôi.',
 'Steve Jobs lớn lên ở Cupertino, California.',
 'Steve Jobs đã phát triển niềm đam mê với công nghệ và điện tử.',
 'Steve Jobs theo học tại Đại học Reed trong một thời gian ngắn.',
 'Steve Jobs đã bỏ học tại Đại học Reed.',
 'Steve Jobs tiếp tục học các khóa học mà mình yêu thích.',
 'Steve Jobs đã học nghệ thuật và thiết kế.',
 'Steve Jobs cùng với Steve Wozniak và Ronald Wayne thành lập Apple Computer, Inc. vào năm 1976.',
 'Steve Jobs, Steve Wozniak và Ronald Wayne đã phát triển chiếc máy tính cá nhân đầu tiên, Apple I.',
 'Apple II là sản phẩm đã đưa Apple trở thành một trong những

In [28]:
len(atomic_facts)

31

In [29]:
labels = df['label'].tolist()
labels

['Supported',
 'Supported',
 'Supported',
 'Supported',
 'Unsupported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported',
 'Supported']

In [30]:
prompts = df['prompt'].to_list()
prompts

['\nBạn được cung cấp một ngữ cảnh (kết quả tìm kiếm web) kèm theo một atomic fact. Nhiệm vụ của bạn là gán nhãn cho atomic fact này là Supported, Unsupported, hoặc Irrelevant. Ngữ cảnh nói về một người hoặc một nhân vật và atomic fact là một thông tin về người đó.\n\nSupported: Chọn nhãn này nếu atomic fact là đúng và được ngữ cảnh hoàn toàn xác thực.\nUnsupported: Chọn nhãn này nếu atomic fact là sai và mâu thuẫn với ngữ cảnh.\nIrrelevant: Chọn nhãn này nếu atomic fact không liên quan đến người hoặc nhân vật, hoặc atomic fact nói về một điều khác không có liên hệ với người đó.\n\nLƯU Ý: thông tin tìm kiếm được trên trang web có thể chứa tin giả và tin sai sự thật. Hãy cân nhắc kĩ lưỡng trước khi ra quyết định của mình.\nĐây là phần bạn cần làm:\nNhân vật: Steve Jobs\nNgữ cảnh từ kết quả tìm kiếm web: [{\'title\': \'Steve Jobs – Wikipedia tiếng Việt\', \'url\': \'https://vi.wikipedia.org/wiki/Steve_Jobs\', \'content\': \'Steve Paul Jobs (/ˈdʒɒbz/; 24 tháng 2 năm 1955 – 5 tháng 10 năm 

In [31]:
# example_response = df['example_response'].to_list()
# example_response

In [32]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")
predicted_labels = []
full_responses = []
correct_prediction = []
for label, atomic_fact,prompt in zip(labels,atomic_facts,prompts):
	print(f"Processing atomic fact: {atomic_fact}")
	print(f"Label: {label}")
	result = get_response_from_prompt(prompt)
	print(result)
	predicted_labels.append(result['label'])
	print(f"Predicted label: {result['label']}")
	full_responses.append(result['full_response'])
	if label==result['label']:
		correct_prediction.append(True)
	else:
		correct_prediction.append(False)

print(correct_prediction)

Processing atomic fact: Steve Jobs là một trong những nhà sáng lập và lãnh đạo nổi bật nhất trong lịch sử ngành công nghệ.
Label: Supported
{'full_response': 'Label: Supported  \nReason: Ngữ cảnh từ kết quả tìm kiếm xác nhận rằng Steve Jobs là một doanh nhân và nhà sáng chế nổi tiếng, đồng sáng lập Apple Inc., và có vai trò quan trọng trong cách mạng máy tính cá nhân, điều này phù hợp với thông tin rằng ông là một trong những nhà sáng lập và lãnh đạo nổi bật nhất trong lịch sử ngành công nghệ.', 'label': 'Supported', 'prompt': '\nBạn được cung cấp một ngữ cảnh (kết quả tìm kiếm web) kèm theo một atomic fact. Nhiệm vụ của bạn là gán nhãn cho atomic fact này là Supported, Unsupported, hoặc Irrelevant. Ngữ cảnh nói về một người hoặc một nhân vật và atomic fact là một thông tin về người đó.\n\nSupported: Chọn nhãn này nếu atomic fact là đúng và được ngữ cảnh hoàn toàn xác thực.\nUnsupported: Chọn nhãn này nếu atomic fact là sai và mâu thuẫn với ngữ cảnh.\nIrrelevant: Chọn nhãn này nếu atom

In [33]:
# query_list = [
# 		"Einstein là một nhà vật lý lý thuyết nổi tiếng người Đức.",
# 		"Einstein được biết đến rộng rãi với những đóng góp đột phá trong lĩnh vực vật lý.",
# 		"Einstein không phải là một học sinh xuất sắc trong trường học."
# 	]
# topic = "Albert Einstein"
# prompt_list = []
# response_list = []
# for query in query_list:
# 	result = get_response(query,topic)
# 	prompt = result['prompt']
# 	response = result['full_response']
# 	prompt_list.append(str(prompt))
# 	response_list.append(response)
# 	print("Query: ",query)
# 	print("Prompt: ",str(prompt))
# 	print("Reponse: ",response)

In [34]:
# Ensemble
# scorers = [ # specify which scorers to include
#     "semantic_negentropy", # black-box scorers
#     llm # use same LLM as a judge
# ]
# uqe = UQEnsemble(llm=llm, scorers=scorers)
# # tune_results = await uqe.tune(
# #     prompts=prompts, ground_truth_answers=example_response
# # )
# # ensemble is now tuned - generate responses on new prompts
# results = await uqe.generate_and_score(prompts=prompts)


# BlackBox UQ
bbuq = BlackBoxUQ(llm=llm, scorers=["semantic_negentropy"], use_best=True, max_calls_per_min=20)

results_BB = await bbuq.generate_and_score(prompts=prompts, num_responses=5)

# LLM as a judge

panel = LLMPanel(llm=llm, judges=[llm,llm,llm,llm,llm], scoring_templates=['true_false_uncertain','true_false_uncertain','true_false_uncertain','true_false_uncertain','true_false_uncertain'])
results_judge = await panel.generate_and_score(prompts=prompts)


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Generating responses...
Generating candidate responses...
Computing confidence scores...
Generating responses...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-ofE4h2u30saTnaQ97U7jYc4b on tokens per min (TPM): Limit 200000, Used 199474, Requested 1258. Please try again in 219ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
results_BB.to_df()

,response,sampled_responses,prompt,semantic_negentropy
0,Label: Supported \nReason: Ngữ cảnh từ kết qu...,[Label: Supported \nReason: Ngữ cảnh xác nhận...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000
1,Label: Unsupported \nReason: Ngữ cảnh khẳng đ...,[Label: Unsupported \nReason: Ngữ cảnh xác đị...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000
2,Label: Supported \nReason: Ngữ cảnh từ các kế...,[Label: Supported \nReason: Ngữ cảnh từ kết q...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.618862
3,Label: Supported \nReason: Ngữ cảnh xác thực ...,[Label: Supported \nReason: Ngữ cảnh xác nhận...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000
4,Label: Supported \nReason: Ngữ cảnh từ kết qu...,[Label: Supported \nReason: Ngữ cảnh xác nhận...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.613147
5,Label: Supported \nReason: Ngữ cảnh xác nhận ...,[Label: Supported \nReason: Ngữ cảnh từ kết q...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000
6,Label: Unsupported \nReason: Ngữ cảnh cho thấ...,[Label: Unsupported \nReason: Ngữ cảnh cho bi...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000
7,Label: Supported \nReason: Ngữ cảnh từ các kế...,[Label: Supported \nReason: Ngữ cảnh xác nhận...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.666099
8,Label: Unsupported \nReason: Ngữ cảnh đã cung...,[Label: Unsupported \nReason: Ngữ cảnh cung c...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.392273
9,Label: Supported \nReason: Ngữ cảnh từ các kế...,[Label: Supported \nReason: Ngữ cảnh từ các k...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.666099


In [ ]:
results_judge.to_df()

,prompt,response,judge_1,judge_2,judge_3,judge_4,judge_5,avg,max,min,median
0,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh xác thực ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Unsupported \nReason: Ngữ cảnh từ các ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh xác nhận ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh từ kết qu...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh từ kết qu...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh xác nhận ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Unsupported \nReason: Ngữ cảnh không x...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh từ kết qu...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Unsupported \nReason: Ngữ cảnh nhấn mạ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported \nReason: Ngữ cảnh cung cấp ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
result_BB = results_BB.to_df()
result_judge = results_judge.to_df()
result =((result_judge['avg'] + result_BB['semantic_negentropy'])/2)>= 0.80

In [ ]:
# result = results.to_df()
# result['ensemble_score'] = result['ensemble_score'] >= 0.90
confidence_list = result.tolist()
for i in range(len(confidence_list)):
	if confidence_list[i]==True and correct_prediction[i]==False:
		print("Wrong atomic fact:", atomic_facts[i])

print(confidence_list)
print(correct_prediction)

Wrong atomic fact: Phạm Nhật Vượng bắt đầu khởi nghiệp bằng cách mở một quán ăn phục vụ người Việt tại Moscow.
[True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [ ]:
count = 0
for v in confidence_list:
	if v == False:
		count+=1
print("Check",count)
print("Total", len(confidence_list))

Check 1
Total 28
